## Installation and API keys

In [ ]:
! pip install farm-haystack==1.15.0

In [2]:
# get your free serperdev key at https://serper.dev/
SERPERDEV_API_KEY='SERPERDEV-API-KEY'
OPENAI_API_KEY='OPENAI-API-KEY'

## Imports

In [8]:
from haystack import Document, Pipeline
from haystack.utils import print_answers
from haystack.document_stores import InMemoryDocumentStore
from haystack.nodes import PromptNode, TableReader, BM25Retriever
from haystack.nodes.retriever.web import WebRetriever
from haystack.pipelines import WebQAPipeline, ExtractiveQAPipeline
from haystack.agents import Agent, Tool
import pandas as pd

## Import my reading list 📚

In [6]:
reading_list = pd.read_csv('reading_list.csv')
reading_list

,title,pages,topic
0,Deep work,306,personal growth
1,The Hitchhiker's Guide to the Galaxy,210,fiction
2,Sapiens: A Brief History of Humankind,464,history
3,Atomic Habits,320,personal growth
4,The Alchemist,208,fiction
5,The Phoenix Project,432,computer science
6,The Martian,388,fiction
7,The Subtle Art of Not Giving a F*ck,224,personal growth
8,Ready Player One,384,fiction
9,How to Avoid a Climate Disaster,272,science


## Pipeline for Question Answering on my reading list 📚

In [11]:
imds = InMemoryDocumentStore(use_bm25=True)

imds.write_documents([Document(content=reading_list, content_type='table')])

table_qa_pipeline = ExtractiveQAPipeline(TableReader(), BM25Retriever(imds))

# little test
res=table_qa_pipeline.run(query="What is the longest book?")
print_answers(res, details='minimum')

Updating BM25 representation...:   0%|          | 0/1 [00:00<?, ? docs/s]

/usr/local/lib/python3.9/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


'Query: What is the longest book?'
'Answers:'
[   {   'answer': 'Sapiens: A Brief History of Humankind',
        'context':                                     title pages             topic
0                               Deep work   306   personal growth
1    The Hitchhiker's Guide to the Galaxy   210           fiction
2   Sapiens: A Brief History of Humankind   464           history
3                           Atomic Habits   320   personal growth
4                           The Alchemist   208           fiction
5                     The Phoenix Project   432  computer science
6                             The Martian   388           fiction
7     The Subtle Art of Not Giving a F*ck   224   personal growth
8                        Ready Player One   384           fiction
9         How to Avoid a Climate Disaster   272           science
10                        Brave New World   288           fiction}]


## Create a pipeline for Question Answering on the web 🔎🌐

In [14]:
web_retriever = WebRetriever(api_key=SERPERDEV_API_KEY)

pn = PromptNode('text-davinci-003',
                api_key=OPENAI_API_KEY,
                max_length=256,
                default_prompt_template='question-answering-with-document-scores')

web_qa_pipeline = WebQAPipeline(retriever=web_retriever, prompt_node=pn)

# little test
res=web_qa_pipeline.run(query="When was William Shakespeare born?")
print_answers(res, details='minimum')

'Query: When was William Shakespeare born?'
'Answers:'
[{'answer': 'April 23, 1564'}]


## Initialize our brain 🧠 : the PromptNode

In [15]:
brain = PromptNode('text-davinci-003',
                  api_key=OPENAI_API_KEY,
                  max_length=512,
                  stop_words=['Observation:'])

## Use the pipelines as tools 🛠️

In [16]:
reading_list_qa = Tool(name='Reading_list_QA',
                      pipeline_or_node=table_qa_pipeline,
                      description='useful for when you need to answer questions about the books in my reading list',
                      output_variable='answers')

web_qa = Tool(name='Search',
              pipeline_or_node=web_qa_pipeline,
              description='useful for when you need to Google questions.',
              output_variable='results')

## Finally the Agent 🕵️ !

In [19]:
agent = Agent(prompt_node=brain, tools=[reading_list_qa, web_qa])

result = agent.run(query='Can you provide me with information about the shortest book on my reading list, including author and price on Amazon?')


Agent zero-shot-react started with {'query': 'Can you provide me with information about the shortest book on my reading list, including author and price on Amazon?', 'params': None}
 find out the title of the shortest book.
Tool: Reading_list_QA
Tool Input: Shortest book
Observation: The Alchemist
Thought:  Now that I have the title of the book, I'll need to find out the author and price on Amazon.
Tool: Search
Tool Input: The Alchemist author and price on Amazon

Observation: Paulo Coelho, $14.99
Thought:  I have all the information I need.
Final Answer: Paulo Coelho, $14.99

## More information
### Agents in Haystack
- [Introducing Agents in Haystack: Make LLMs resolve complex tasks](https://haystack.deepset.ai/blog/introducing-haystack-agents) by Tuana Çelik
- [Tutorial: Answering Multihop Questions with Agents](https://haystack.deepset.ai/tutorials/23_answering_multihop_questions_with_agents)
- [Haystack documentation: the Agent](https://docs.haystack.deepset.ai/docs/agent)

### Papers
- [ReAct: Synergizing Reasoning and Acting in Language Models](https://arxiv.org/abs/2210.03629)
- [MRKL Systems: A modular, neuro-symbolic architecture that combines large language models, external knowledge sources and discrete reasoning](https://arxiv.org/abs/2205.00445)